<a href="https://colab.research.google.com/github/jesusmlb/sales_forecast/blob/main/demand_forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predict the sales units of individual products

In [ ]:
# Data Preprocessing
import pandas as pd
import numpy as np

# Data Visualisation
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import seaborn as sns
import plotly.express as px

# Modelling and Forecasting
import sklearn
import skforecast
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.recursive import ForecasterRecursive
from skforecast.model_selection import TimeSeriesFold, OneStepAheadFold
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import bayesian_search_forecaster
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection import bayesian_search_forecaster_multiseries
from skforecast.feature_selection import select_features_multiseries
from skforecast.preprocessing import RollingFeatures
from skforecast.exceptions import OneStepAheadValidationWarning
from skforecast.plot import set_dark_theme
from skforecast.preprocessing import series_long_to_dict
from skforecast.preprocessing import exog_long_to_dict

# Warnings configuration
import warnings

#### Let's import our dataset and have a look at the shape

In [ ]:
# Let's import our dataset
df = pd.read_csv('MasterDataTEST_OP.csv', low_memory=False)

# Let's check the shape of our dataset
df.shape

#### Let's check our dataset info

In [ ]:
df.info()

#### Before changing the datatype of our date columns, let's examine why there are so many null values.

In [ ]:
# Let's check the whole row with non-null values of next_delivery_date_1
df[df['next_delivery_date_1'].isna()].describe()

#### It is interesting that we don't have delivery dates for next_delivery_date_1, yet we have quantities to be delivered. How many such cases do we have?

In [ ]:
df[(df['next_delivery_quantity_1'] > 0) & (df['next_delivery_date_1'].isna())].shape

In [ ]:
# Let's see the rows and evaluate the sku with nan values in dates
df[(df['next_delivery_quantity_1'] > 0) & (df['next_delivery_date_1'].isna())]

In [ ]:
# Let's check how many weeks of weeks_to_next_delivery_1 for the product "UWYXC" usually has
df[df['sku_id'] == 'UWYXC'].iloc[:, :20]

After checking the items in delivery_date_1 with null values, we recognized that delivery dates typically start when the order is placed. Therefore, we can fill these null values with the number of days the order has been waiting to arrive.

For the other NaN values, we understand that they occur because there are no quantities to be delivered.

In [ ]:
# Function to update 'weeks_to_next_delivery_1' for filtered rows in each SKU
def fill_weeks_to_next_delivery(group, col):
    # Define the filter condition within the group
    group_condition = (group[f'next_delivery_quantity_{col}'] > 0) & (group[f'next_delivery_date_{col}'].isna())
    filtered_group = group[group_condition]
    if not filtered_group.empty:
        # Determine the number of rows matching the condition
        num_rows = len(filtered_group)
        # Create a countdown sequence from num_rows to 1 for filtered rows
        group.loc[filtered_group.index, f'weeks_to_next_delivery_{col}'] = range(num_rows, 0, -1)
    return group

In [ ]:
# Apply the function to each SKU group
df = df.groupby('sku_id', group_keys=False).apply(fill_weeks_to_next_delivery, col=1)

In [ ]:
# Let's see some rows to see the results of the functions
df[(df['next_delivery_quantity_1'] > 0) & (df['next_delivery_date_1'].isna())].head()

##### Great now let's add the dates to next_delivery_date_1

In [ ]:
# Because our cal_date is not in datetime format, we need to convert it first
df['cal_date'] = pd.to_datetime(df['cal_date'])

# Fill the next_delivery_date_1 with cal_date + weeks_to_next_delivery_1 but just dates without timestamp
df['next_delivery_date_1'] = df['next_delivery_date_1'].fillna(df['cal_date'] + pd.to_timedelta(df['weeks_to_next_delivery_1'], unit='W'))

In [ ]:
# Let's have a look again to the null values
df.info()

In [ ]:
# Now let's do the same for the rest of the dates_dalivery
for i in range(2, 6):
    df = df.groupby('sku_id', group_keys=False).apply(fill_weeks_to_next_delivery, col=i)
    df[f'next_delivery_date_{i}'] = df[f'next_delivery_date_{i}'].fillna(df['cal_date'] + pd.to_timedelta(df[f'weeks_to_next_delivery_{i}'], unit='W'))

In [ ]:
df.info()

#### Great, now that we have the data complete. Let's change the data type of the dates columns

In [ ]:
# Convert the next_delivery_date columns to datetime
date_columns = [f'next_delivery_date_{i}' for i in range(1, 6)]
df[date_columns] = df[date_columns].apply(pd.to_datetime)

In [ ]:
# Let's check again the datatypes of our columns
df.info()

#### Let's check duplicates

In [ ]:
# Let's sum the duplicates in the dataset
df.duplicated().sum()

In [ ]:
# Let's see the range of our cal_date
df['cal_date'].min(), df['cal_date'].max()

In [ ]:
# And how many sku do we have in the dataset
df['sku_id'].nunique()

In [ ]:
# Let's create a basic summary of the dataset
df.describe()

#### The first thing we noticed was the presence of negative values in gsales_v. This could be due to several reasons, such as product returns, inventory adjustments, or data entry errors. We will examine the quantity of negative values in gsales_v and compare them with the inventory data to validate the first hypothesis.

In [ ]:
# Why do we have negative gsales_v? Let's have a look
df[df['gsales_v'] < 0]

In [ ]:
# First we can see that the quantities (gsales_u) is 0. Let's pick one item to see the details

df[df['sku_id'] == 'QCLLY'].iloc[40:80, [0, 1, 2, 3, 4]] # Let's pick just the columns we are interested in

#### We observe that after a negative sales value, the inventory shows an increase in the quantity of the product. This indicates a product return. We can keep these rows. Next, let's examine other products with negative sales values.

In [ ]:
# Group the data by SKU and then process each group to find comparisons
grouped_comparisons = []

for sku, group in df.groupby('sku_id'):
    # Identify rows with negative `gsales_v` within the group
    negative_gsales_v_group = group[group['gsales_v'] < 0]
    # Get the previous rows for the negative `gsales_v` rows
    previous_rows_group = group.shift(1).loc[negative_gsales_v_group.index]
    # Combine for the group and append the result
    comparison_group = pd.concat([previous_rows_group, negative_gsales_v_group], keys=['Previous', 'Current'])
    comparison_group['sku_id'] = sku  # Ensure SKU is included
    grouped_comparisons.append(comparison_group)

# Combine all grouped comparisons into a single DataFrame
grouped_comparisons_df = pd.concat(grouped_comparisons)


In [ ]:
# Let's have a look at the grouped_comparisons_df
grouped_comparisons_df.iloc[40:80, [0, 1, 2, 3, 4, 26, 27, 28, 29, 30, 33, 34, 35]]

##### After examining other products, we recognized that not all of them follow the same logic of returns; some lack sales quantities or promotions applied. We can drop these rows from the analysis. It would be worthwhile to discuss with the business team to understand the reasons behind these negative values.

In [ ]:
# Filter rows where `gsales_v` is negative
negative_gsales_v = df[df['gsales_v'] < 0]

# Get the previous rows for the negative `gsales_v` DataFrame
previous_rows = negative_gsales_v.groupby('sku_id').shift(1)

# Compare `dc_reg_inventory_u` between current and previous rows for negative `gsales_v` rows
rows_without_change = negative_gsales_v[
    (negative_gsales_v['dc_reg_inventory_u'] == previous_rows['dc_reg_inventory_u'])
]

# Get the indices of rows with changes
indices_without_change = rows_without_change.index

# Let's drop from the dataset the rows with
print(df.shape)
df = df.drop(indices_without_change)
print(df.shape)

#### Great now that we are ok with the negative value. Let's check the dc_reg_inventory_u and check why we are having negative values in the inventory.

In [ ]:
df[df['dc_reg_inventory_u'] < 0]

#### We observed that gsales_u has positive values, which could indicate that the product was sold but the system did not update the inventory (backorders). We will replace these negative values with zero, as we aim to preserve the product demand for forecasting purposes.

In [ ]:
# Replace negative values in 'dc_reg_inventory_u' with 0
df['dc_reg_inventory_u'] = df['dc_reg_inventory_u'].clip(lower=0)

In [ ]:
df.describe()

#### Our next column will be weeks_to_next_delivery_1 and _2 to. Let's see what happened with them.

In [ ]:
# Let's filter those sku_id with weeks_to_next_delivery_1 that are negative
df[df['weeks_to_next_delivery_1'] < 0]["sku_id"].unique()

In [ ]:
# Let's check this sku_id = OPCIZ
df[df['sku_id'] == 'UYZCL'].iloc[60:, [0, 1, 2, 3, 4, 5, 6, 7]]

#### These negative values are occurring due to overdue deliveries. We should retain these values to better understand the lead times of our suppliers and improve our demand planning process.

In [ ]:
df.info()

## Exploratory Data Analysis

#### Since our goal is to forecast the next 8 weeks, let's keep our dataset until 31-12-2023 to ensure a realistic scenario for the analysis and forecast.

In [ ]:
# Let's have our dataset until 31-12-2023
df = df[df['cal_date'] <= '2023-12-31']
df.shape

In [ ]:
import nbformat
print(nbformat.__version__)

In [ ]:
# How is our demand overtime?
fig = px.line(df.groupby('cal_date')['gsales_u'].sum().reset_index(), x='cal_date', y='gsales_u', title='Total Demand Over Time')
fig.update_layout(xaxis_title='Date', yaxis_title='Total Demand', template='plotly_white', xaxis_showgrid=False, yaxis_showgrid=False)
fig.show()

#### There are remarkable peaks in the sales of some products. Let's check those weeks in specific.

In [ ]:
df.groupby('cal_date')['gsales_u'].sum().sort_values(ascending=False).head(10)

In [ ]:
# Let's filter these dates to see products and compare on_promo column
days = ["2023-11-26", "2022-11-27", "2021-11-28"]

others_days = ["2023-08-15", "2023-02-05", "2021-08-06"]

# Filter thes edays and check the other columsn
print(df[df['cal_date'].isin(days)].iloc[:, 26:]["on_promo"].value_counts())
print(df[df['cal_date'].isin(others_days)].iloc[:, 26:]["on_promo"].value_counts())

#### After comparing, we can see that our promotions are having a significant impact on sales. These promotions coincide with Black Friday week, which explains the increase in sales.

#### What if we analyze sales trends across the months and weeks of the year and evaluate the highest and lowest sales periods?

In [ ]:
# Calculate average weekly/monthly gsales_u
df['week'] = df['cal_date'].dt.isocalendar().week
df['month'] = df['cal_date'].dt.month

# Calculate average weekly gsales_u
weekly_gsales_u = df.groupby('week')['gsales_u'].mean()

# Calculate average monthly gsales_u
monthly_gsales_u = df.groupby('month')['gsales_u'].mean()

# Plot the average weekly gsales_u
fig = px.line(weekly_gsales_u, title='Average Weekly Demand')
fig.update_layout(xaxis_title='Week', yaxis_title='Average Demand', template='plotly_white', xaxis_showgrid=False, yaxis_showgrid=False)
fig.show()

# Plot the average monthly gsales_u
fig = px.line(monthly_gsales_u, title='Average Monthly Demand')
fig.update_layout(xaxis_title='Month', yaxis_title='Average Demand', template='plotly_white', xaxis_showgrid=False, yaxis_showgrid=False)
fig.show()

In [ ]:
# Let's plot the sales over months but change the colour for each year, so we can identify any pattern
df['year'] = df['cal_date'].dt.year

fig = px.line(df.groupby(['year', 'month'])['gsales_u'].sum().reset_index(), x='month', y='gsales_u', color='year', title='Total Demand Over Months')
fig.update_layout(xaxis_title='Month', yaxis_title='Total Demand', template='plotly_white', xaxis_showgrid=False, yaxis_showgrid=False)
fig.show()

# Let's plot the sales over weeks but change the colour for each year, so we can identify any pattern
fig = px.line(df.groupby(['year', 'week'])['gsales_u'].sum().reset_index(), x='week', y='gsales_u', color='year', title='Total Demand Over Weeks')
fig.update_layout(xaxis_title='Week', yaxis_title='Total Demand', template='plotly_white', xaxis_showgrid=False, yaxis_showgrid=False)
fig.show()

#### It is difficult to identify seasonality in the weekly data, but when looking at the monthly data, we observe that months 2 and 3 show lower sales, followed by an increase in sales after these months. There is another increase in months 8 and 11 due to promotions. Let's compare the number of promotions across the months.

In [ ]:
# Let's plot the count of promotions over months but change the colour for each year, so we can identify any pattern
fig = px.line(df.groupby(['year', 'month'])['on_promo'].sum().reset_index(), x='month', y='on_promo', color='year', title='Total Promotions Over Months')
fig.update_layout(xaxis_title='Month', yaxis_title='Total Promotions', template='plotly_white', xaxis_showgrid=False, yaxis_showgrid=False)
fig.show()

In [ ]:
# Let's count on_mkdn
fig = px.line(df.groupby(['year', 'month'])['on_mkdn'].sum().reset_index(), x='month', y='on_mkdn', color='year', title='Total Markdowns Over Months')
fig.update_layout(xaxis_title='Month', yaxis_title='Total Markdowns', template='plotly_white', xaxis_showgrid=False, yaxis_showgrid=False)
fig.show()

#### Markdown prices have become more prevalent in recent years. We observed that after August 2021, they started to occur more regularly and are now active year-round. For the on_promo column, we only see this event during the months of November and December. This column will have a significant impact on our forecast and should be included as a predictor.

In [ ]:
#### Let's have a look on the items alone and evaluate the time series
items = ['QCLLY', 'UYZCL', 'UWYXC', 'QWYXC', 'QWYXC']

for item in items:
    fig = px.line(df[df['sku_id'] == item], x='cal_date', y='gsales_u', title=f'Trend of Sales for {item}')
    fig.update_layout(xaxis_title='Date', yaxis_title='Units Sold', template='plotly_white', xaxis_showgrid=False, yaxis_showgrid=False)
    fig.show()

#### Interesting wer have different lenght of time series and some of our products don't have sales. Let's check the products with no sales

In [ ]:
# Let's find those sku that the sum of gsales_u is 0
sum_of_items = df.groupby('sku_id')['gsales_u'].sum().sort_values().reset_index()

# Let's filter those with 0 sales
zero_sales = sum_of_items[sum_of_items['gsales_u'] == 0]['sku_id'].tolist()

In [ ]:
# How many products in our dataset are with zero sales?
#zero_sales = df[(df['gsales_u'] == 0) & (df['dc_reg_inventory_u'] == 0)]

# Let's see the percentage of zero sales
zero_sales_percentage = (len(zero_sales) / df["sku_id"].nunique()) * 100

print(f'The percentage of products with zero sales is {zero_sales_percentage:.2f}%')

### Wow, I wasn't expecting that much. Are this product Out of stock? Let's check the inventory of these products.

In [ ]:
df[df['sku_id'].isin(zero_sales)].iloc[:, [2, 5]].describe()

#### We are going to place these items in a separate dataset and discuss them with the client. We have products with zero inventory that may need to be cleared or delisted, and products with inventory that may require a discussion on whether we need to apply some kind of promotion to sell them.

#### This may be due to data entry issues. We will investigate further. For our modeling purposes, we will drop these rows.

In [ ]:
# remove the items with 0 sales
print(df.shape)
df = df[~df['sku_id'].isin(zero_sales)]
print(df.shape)

In [ ]:
#### Let's have a look on the items alone and evaluate the time series
items = np.random.choice(df['sku_id'].unique(), 5)

for item in items:
    fig = px.line(df[df['sku_id'] == item], x='cal_date', y='gsales_u', title=f'Trend of Sales for {item}')
    fig.update_layout(xaxis_title='Date', yaxis_title='Units Sold', template='plotly_white', xaxis_showgrid=False, yaxis_showgrid=False)
    fig.show()

In [ ]:
df[df['sku_id'] == 'MENEX']

#### We might have a look on those item with just 1 sale

In [ ]:
sum_1_sales = df.groupby('sku_id')['gsales_u'].sum().sort_values().reset_index()

# Let's filter those witt less than 5 sales
one_sold = sum_1_sales[sum_1_sales['gsales_u'] == 1]['sku_id'].tolist()

len(one_sold)

#### Our model won't be able to capture any information with these products, we are going to remove them from our dataset.

In [ ]:
# Remove the items with 1 sale
print(df.shape)
df = df[~df['sku_id'].isin(one_sold)]
print(df.shape)

In [ ]:
#### Let's have a look on the items alone and evaluate the time series
items = np.random.choice(df['sku_id'].unique(), 5)

for item in items:
    fig = px.line(df[df['sku_id'] == item], x='cal_date', y='gsales_u', title=f'Trend of Sales for {item}')
    fig.update_layout(xaxis_title='Date', yaxis_title='Units Sold', template='plotly_white', xaxis_showgrid=False, yaxis_showgrid=False)
    fig.show()

#### Now that we have at least 2 sales for each product. Let's check the correlation between the columns

In [ ]:
# Let's plot a heatmap for the correlation matrix between our numerical values
numeric_df = df.select_dtypes(include=[np.number])
corr = numeric_df.corr()
fig, ax = plt.subplots(figsize=(26, 15))
sns.heatmap(corr, annot=True, cmap='coolwarm', ax=ax)
plt.title('Correlation Matrix')
plt.show()

#### We observe that a few columns have a higher positive correlation with gsales_u, such as gsales_v, inventory, and next_delivery_quantity, which were expected. Even though our sales increased significantly in the last months of the year due to promotions, the algorithm did not capture this information. What else might we be missing? Let's plot the two variables.

In [ ]:
# Let's plot gsales_u and on_promo scatter plot and colour the dots with on_mkdn
fig = px.scatter(df, x='gsales_u', y='on_promo', color='on_mkdn', title='Scatter Plot of units sold and on_promo with on_mkdn')
fig.update_layout(xaxis_title='gsales_u', yaxis_title='on_promo', template='plotly_white')
fig.show()

In [ ]:
# How much percentage represents thos sales in promo and markdowns over the total units sold?

# Print the percentage of sales in promo and markdowns over the total units sold
promo_percentage = df['on_promo'].value_counts(normalize=True) * 100
mkdn_percentage = df['on_mkdn'].value_counts(normalize=True) * 100

print("Percentage of sales in promo:")
print(promo_percentage)
print("\nPercentage of sales in markdowns:")
print(mkdn_percentage)

#### Markdown represent a considerable percentage of the sales. Interesgintly there is no correlation between on_mkdn and gsales_u.

#### Let's have a look on the categories and subcategories of the products

In [ ]:
df.columns

In [ ]:
df['division_desc'].value_counts().reset_index()

In [ ]:
# Let's plot our sales by division_desc, department_desc and category_desc
fig = px.bar(df['division_desc'].value_counts().reset_index(), x='division_desc', y='count', title='Total Sales by Division')
fig.update_layout(xaxis_title='Division', yaxis_title='Total Sales', template='plotly_white', xaxis_showgrid=False, yaxis_showgrid=False)
fig.show()

fig = px.bar(df['department_desc'].value_counts().reset_index(), x='department_desc', y='count', title='Total Sales by Department')
fig.update_layout(xaxis_title='Department', yaxis_title='Total Sales', template='plotly_white', xaxis_showgrid=False, yaxis_showgrid=False)
fig.show()

fig = px.bar(df['category_desc'].value_counts().reset_index(), x='category_desc', y='count', title='Total Sales by Category')
fig.update_layout(xaxis_title='Category', yaxis_title='Total Sales', template='plotly_white', xaxis_showgrid=False, yaxis_showgrid=False)
fig.show()

In [ ]:
# Let's group wich department_desc has the most products in promo and markdowns
promo_category = df[df['on_promo'] == 1]['department_desc'].value_counts()
mkdn_category = df[df['on_mkdn'] == 1]['department_desc'].value_counts()

print("Category with most products in promo:")
print(promo_category.head(10))
print("\nCategory with most products in markdowns:")
print(mkdn_category.head(10))

In [ ]:
# Now let's compare the units sold in percentage. How much sales were made with 0 on_promo and 0 on_mkdn? So we can see how much drive has this initiatives
# Calculate the percentage of sales with 0 on_promo and 0 on_mkdn
total_sales = df['gsales_u'].sum()
sales_no_promo_no_mkdn = df[(df['on_mkdn'] == 0)]['gsales_u'].sum()

percentage_no_promo_no_mkdn = (sales_no_promo_no_mkdn / total_sales) * 100

print(f"Percentage of sales with 0 on_promo and 0 on_mkdn: {percentage_no_promo_no_mkdn:.2f}%")
# Calculate the percentage of sales with 0 on_promo and 0 on_mkdn for each department_desc
department_sales = df.groupby('department_desc')['gsales_u'].sum()
department_sales_no_promo_no_mkdn = df[(df['on_mkdn'] == 0)].groupby('department_desc')['gsales_u'].sum()

percentage_no_promo_no_mkdn_by_department = (department_sales_no_promo_no_mkdn / department_sales) * 100

print("Percentage of sales with 0 on_promo and 0 on_mkdn by department:")
print(percentage_no_promo_no_mkdn_by_department)

#### We observe that there is a low percentage of 1 values in the on_mkdn column across departments, so we conclude that promotions and price reductions will not be significant drivers for sales. Therefore, we need to continue exploring other variables.

In [ ]:
# let's examine the inventory levels over time
fig = px.line(df.groupby('cal_date')['dc_reg_inventory_u'].sum().reset_index(), x='cal_date', y='dc_reg_inventory_u', title='Total Inventory Over Time')
fig.update_layout(xaxis_title='Date', yaxis_title='Total Inventory', template='plotly_white', xaxis_showgrid=False, yaxis_showgrid=False)
fig.show()

#### This aligned with the sale, where the planner prepare for the seasonality of the last months. Let's have a look on the distribution of the inventory.

In [ ]:
# Let's plot eh distribution of the inventory levels
fig = px.histogram(df, x='dc_reg_inventory_u', title='Distribution of Inventory Levels')
fig.update_layout(xaxis_title='Inventory Level', yaxis_title='Count', template='plotly_white')
fig.show()

In [ ]:
# Is there any relation with the gsales_u and the inventory levels?
fig = px.scatter(df, x='dc_reg_inventory_u', y='gsales_u', title='Scatter Plot of Inventory Levels and Units Sold')
fig.update_layout(xaxis_title='Inventory Level', yaxis_title='Units Sold', template='plotly_white')
fig.show()

In [ ]:
from plotly.subplots import make_subplots

import plotly.graph_objects as go

# Create a figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add trace for inventory levels
fig.add_trace(
    go.Scatter(x=df.groupby('cal_date')['dc_reg_inventory_u'].sum().index,
               y=df.groupby('cal_date')['dc_reg_inventory_u'].sum(),
               name="Total Inventory"),
    secondary_y=False,
)

# Add trace for total sales
fig.add_trace(
    go.Scatter(x=df.groupby('cal_date')['gsales_u'].sum().index,
               y=df.groupby('cal_date')['gsales_u'].sum(),
               name="Total Sales"),
    secondary_y=True,
)

# Add figure title and axis labels
fig.update_layout(
    title_text="Total Inventory and Sales Over Time"
)

fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Total Inventory", secondary_y=False)
fig.update_yaxes(title_text="Total Sales", secondary_y=True)

# Show the plot
fig.show()

#### Interesting how the inbound inventory levels are 3-4 months before the sales. Is that the common lead time? Let's have a look on the concatenate all weeks_to_next_delivery_ columns and see the distribution

In [ ]:
# Interesting how the inbound inventory levels are 3'4 months before the sales. Is that the common lead time? Let's have a look on the concatenate all weeks_to_next_delivery_ columns and see the distribution
weeks_to_next_delivery = pd.concat([df['weeks_to_next_delivery_1'], df['weeks_to_next_delivery_2'], df['weeks_to_next_delivery_3'], df['weeks_to_next_delivery_4'], df['weeks_to_next_delivery_5']])
weeks_to_next_delivery = weeks_to_next_delivery[weeks_to_next_delivery != 0]
fig = px.histogram(weeks_to_next_delivery, title='Distribution of Weeks to Next Delivery')
fig.update_layout(xaxis_title='Weeks to Next Delivery', yaxis_title='Count', template='plotly_white')
fig.show()

# Check later


In [ ]:
df.columns

#### Let's have a look on the oo_inventory_u if we can find any patterns

In [ ]:
fig = px.line(df.groupby('cal_date')['oo_inventory_u'].sum().reset_index(), x='cal_date', y='oo_inventory_u', title='Total OO Inventory Over Time')
fig.update_layout(xaxis_title='Date', yaxis_title='Total OO Inventory', template='plotly_white', xaxis_showgrid=False, yaxis_showgrid=False)
fig.show()

#### yep, this is aligned with our inventory levels

#### Now we can move to the prices and see how they are distributed

In [ ]:
# Let's plot the distribution of the full_price
fig = px.histogram(df, x='full_price', title='Distribution of Full Price')
fig.update_layout(xaxis_title='Full Price', yaxis_title='Count', template='plotly_white')
fig.show()
# Plot the distribution of the current_price
fig = px.histogram(df, x='current_price', title='Distribution of Current Price')
fig.update_layout(xaxis_title='Current Price', yaxis_title='Count', template='plotly_white')
fig.show()

# Plot the distribution of the ticket_price
fig = px.histogram(df, x='ticket_price', title='Distribution of Ticket Price')
fig.update_layout(xaxis_title='Ticket Price', yaxis_title='Count', template='plotly_white')
fig.show()

#### As expected we have the lower prices in markdowns. Let's check the distribution of the prices in the products

In [ ]:
# Now let's filter those product in on_promo 1 and on_mkdn 1 and see the distribution of the prices
fig = px.histogram(df[(df['on_promo'] == 1) & (df['on_mkdn'] == 1)], x='full_price', title='Distribution of Full Price for Products in Promo and Markdowns')
fig.update_layout(xaxis_title='Full Price', yaxis_title='Count', template='plotly_white')
fig.show()

#### We can confirm that our prices are reduced by the promotions. But how effective are these promotions in gsales_v?

In [ ]:
# Calculate the average sales during promotion periods
promo_sales = df[df['on_promo'] == 1]['gsales_u'].mean()

# Calculate the average sales during non-promotion periods
non_promo_sales = df[df['on_promo'] == 0]['gsales_u'].mean()

# Calculate the percentage increase in sales during promotion periods
percentage_increase = ((promo_sales - non_promo_sales) / non_promo_sales) * 100

print(f"Average sales during promotion periods: {promo_sales:.2f}")
print(f"Average sales during non-promotion periods: {non_promo_sales:.2f}")
print(f"Percentage increase in sales during promotion periods: {percentage_increase:.2f}%")

#### After this comparison, we can confirm and recommend advising the business to inform us whenever they plan to run a promotion, so we can improve the forecast for those periods. The percentage increase in sales is a key factor for our analysis and provides valuable insights for the forecast.

## Feature Engineering

#### We already have month and week. Let's add if the week is the last week of the month. This could be a driver for the sales.

In [ ]:
# Let's add if the week is the last of the month or not
df['last_week_of_month'] = df['cal_date'].dt.is_month_end

# Because we have the size code, maybe we are deailing with clothes. Let's add the season column
df['season'] = df['cal_date'].dt.quarter

#### Price feature engineering

In [ ]:
# Let's add the discount and the discount percentage. This will give us an idea if the percentage of discount is related to the sales
df['price_discount'] = df['full_price'] - df['current_price']
df['discount_percentage'] = (df['price_discount'] / df['full_price'])

In [ ]:
# Now let's add the promotion days is going to be that product
df['total_discount_days'] = df['promo_days_this_week'] + df['mkdn_days_this_week']

### Inventory feature engineering

In [ ]:
# Inventory ratios
df['total_incoming_inventory'] = (df['next_delivery_quantity_1'] +
                                df['next_delivery_quantity_2'] +
                                df['next_delivery_quantity_3'] +
                                df['next_delivery_quantity_4'] +
                                df['next_delivery_quantity_5'])

#### Categorical feture engineering

##### On this one we have 4, division_desc, department_desc, category_desc and size_code. We are going to use the get_dummies to transform these columns into numerical values.

In [ ]:
# let's encode our categorical columns
df = pd.get_dummies(df, columns=['division_desc', 'department_desc', 'category_desc', 'size_code'], drop_first=True)

In [ ]:
df.columns

In [ ]:
# Now let's evaluate again the correlation matrix

numeric_df_2 = df.select_dtypes(include=[np.number])
corr = numeric_df_2.iloc[:, :44].corr()
fig, ax = plt.subplots(figsize=(26, 15))
sns.heatmap(corr, annot=True, cmap='coolwarm', ax=ax)
plt.title('Correlation Matrix')
plt.show()


#### Because our linear correlation doesn't provide additional insights, let's create a simple decision tree model to see if we can capture any non-linear relationships between the columns.

In [ ]:
# Define the features (X) and the target variable (y)
# Because sales are usually corrlated with the numbers of units sold, I am going to exclude from the basic regression model to evaluate the new features
X = df.drop(columns=['gsales_u', 'gsales_v', 'sku_id', 'cal_date', 'next_delivery_date_1', 'next_delivery_date_2', 'next_delivery_date_3', 'next_delivery_date_4', 'next_delivery_date_5'])
y = df['gsales_u']

# Feature importance (using a simple model)
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()

rf.fit(X, y)

feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print(feature_importance)


#### That't interesting that our inventory is one of the important for our independent variables. Let's filter just those one grater than 0.015

In [ ]:
# Let's print out the filter
print(feature_importance[feature_importance['importance'] > 0.015])


#### This aligns with the previous plot we created between sales and inventory. Our lead times and inventory increases serve as indicators that we are entering the season when our sales increase.

In [ ]:
# Let's save in a list the features importance that are higher than 0.015
selected_features = feature_importance[feature_importance['importance'] > 0.015]['feature'].tolist()

# Now let's filter our dataset with different name, needs to include as well the target variable, dates and sku_id
df_filtered = df[['gsales_u', 'sku_id', 'cal_date'] + selected_features]

In [ ]:
df_filtered.info()

#### We can see that week and category has different dtypes, let's change them.

In [ ]:
# Week and month should be integer64
df_filtered.loc[:, 'week'] = df_filtered['week'].astype('int64')
df_filtered.loc[:, 'month'] = df_filtered['month'].astype('int64')
df_filtered.loc[:, 'year'] = df_filtered['year'].astype('int64')

In [ ]:
# Let's plot the scatter plot of each feature with the target variable
for feature in selected_features:
    fig = px.scatter(df_filtered, x=feature, y='gsales_u', title=f'Scatter Plot of {feature} and Units Sold')
    fig.update_layout(xaxis_title=feature, yaxis_title='Units Sold', template='plotly_white')
    fig.show()

#### After evaluating our features, none of them show statistical significance for our model. We are now going to try incorporating lag features from our target variable and assess their importance.

#### For this case, that we have many time series we are going to model univartie time series and multivariate time series.

#### But first let's create our benchmark model. We are going to use the mean of the sales of the last 8 weeks.

In [ ]:
import pandas as pd

# Ensure the 'cal_date' column is in datetime format
df_filtered['cal_date'] = pd.to_datetime(df_filtered['cal_date'])

# Create the moving average for the past 8 weeks, shifted to exclude the current week
df_filtered['mean_8_weeks'] = df_filtered.groupby('sku_id')['gsales_u'].transform(
    lambda x: x.shift(1).rolling(8, min_periods=1).mean()
)

# Split into train and test datasets
train = df_filtered[df_filtered['cal_date'] <= '2023-11-12']
test = df_filtered[df_filtered['cal_date'] > '2023-11-12']

# Extract the last row of training data for each SKU (latest known mean_8_weeks value)
train_last_means = train.groupby('sku_id')[['mean_8_weeks']].last().reset_index()

# Prepare the test set for the next 8 weeks
test_8_weeks = pd.DataFrame()
start_date = pd.to_datetime('2023-11-12')
for i in range(1, 9):
    # Generate the date for each week
    target_date = start_date + pd.DateOffset(weeks=i)
    week_data = test[test['cal_date'] == target_date]

    if not week_data.empty:
        test_8_weeks = pd.concat([test_8_weeks, week_data], axis=0)

# Merge the last means from train into the test_8_weeks dataset
test_8_weeks = test_8_weeks.merge(train_last_means, on='sku_id', how='left')

# Compare the benchmark predictions with actual sales
test_8_weeks['benchmark_error'] = (test_8_weeks['gsales_u'] - test_8_weeks['mean_8_weeks_y']).abs()

# Evaluate the benchmark model (e.g., using Mean Absolute Error)
mae_benchmark = test_8_weeks['benchmark_error'].mean()

print(f"Mean Absolute Error of the benchmark model: {mae_benchmark}")

In [ ]:
sku = np.random.choice(test_8_weeks['sku_id'].unique(), 5)

for item in sku:
    fig = make_subplots(rows=1, cols=1)
    fig.add_trace(go.Scatter(x=test_8_weeks[test_8_weeks['sku_id'] == item]['cal_date'], y=test_8_weeks[test_8_weeks['sku_id'] == item]['gsales_u'], name='Actual', mode='lines', marker=dict(color='blue')), row=1, col=1)
    fig.add_trace(go.Scatter(x=test_8_weeks[test_8_weeks['sku_id'] == item]['cal_date'], y=test_8_weeks[test_8_weeks['sku_id'] == item]['mean_8_weeks_y'], name='Predicted', mode='lines', marker=dict(color='red')), row=1, col=1)
    fig.update_layout(title=f'Actual vs Predicted for {item}', xaxis_title='Date', yaxis_title='Units Sold', template='plotly_white')
    fig.show()


#### Now that we have our benchmark, let's go with the univariate time series and multivariate time series to evaluate if we can beat the benchmark. 16.58%

#### We are going to use the library Skforecast who can help us to achieve this

In [ ]:


color = '\033[1m\033[38;5;208m'
print(f"{color}Version skforecast: {skforecast.__version__}")
print(f"{color}Version scikit-learn: {sklearn.__version__}")
print(f"{color}Version pandas: {pd.__version__}")
print(f"{color}Version numpy: {np.__version__}")

In [ ]:
# Data preprocessing
# ======================================================================================
selected_items = df_filtered.sku_id.unique()
data = df_filtered[(df_filtered['sku_id'].isin(selected_items))].copy()
data['date'] = pd.to_datetime(data['cal_date'], format='%Y-%m-%d')
data = pd.pivot_table(
           data    = data,
           values  = 'gsales_u',
           index   = 'date',
           columns = 'sku_id'
       )
data.columns.name = None
data.columns = [f"{col}" for col in data.columns]
data = data.asfreq('1W')
data = data.sort_index()
# replace NaN values with 0
data = data.fillna(0)
data.head(4)

In [ ]:
# Split data into train-validation-test
# ======================================================================================
end_train = '2023-09-17'
end_val = '2023-11-12'
data_train = data.loc[:end_train, :].copy()
data_val   = data.loc[end_train:end_val, :].copy()
data_test  = data.loc[end_val:, :].copy()
print(f"Train dates      : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})")
print(f"Validation dates : {data_val.index.min()} --- {data_val.index.max()}  (n={len(data_val)})")
print(f"Test dates       : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})")

In [ ]:
# Plot time series
# ======================================================================================
set_dark_theme()
fig, axs = plt.subplots(4, 1, figsize=(7, 5), sharex=True)
data.iloc[:, :4].plot(
    legend   = True,
    subplots = True,
    title    = 'Unis sales',
    ax       = axs,
)
for ax in axs:
    ax.axvline(pd.to_datetime(end_train) , color='white', linestyle='--', linewidth=1.5)
    ax.axvline(pd.to_datetime(end_val) , color='white', linestyle='--', linewidth=1.5)
fig.tight_layout()
plt.show()

In [ ]:
# Train and backtest a model for each item: ForecasterAutoreg
items = []
mae_values = []
predictions = {}

for i, item in enumerate(tqdm(data.columns)):
    # Define forecaster
    window_features = RollingFeatures(stats=['mean', 'min', 'max'], window_sizes=4)
    forecaster = ForecasterRecursive(
                     regressor       = HistGradientBoostingRegressor(random_state=42),
                     lags            = 2,
                     window_features = window_features
                 )
    # Backtesting forecaster
    cv = TimeSeriesFold(
            steps              = 1,
            initial_train_size = len(data_train) + len(data_val),
            refit              = False,
         )
    metric, preds = backtesting_forecaster(
                        forecaster    = forecaster,
                        y             = data[item],
                        cv            = cv,
                        metric        = 'mean_absolute_error',
                        verbose       = False,
                        show_progress = False
                    )
    items.append(item)
    mae_values.append(metric.at[0, 'mean_absolute_error'])
    predictions[item] = preds

# Results
uni_series_mae = pd.Series(
                     data  = mae_values,
                     index = items,
                     name  = 'uni_series_mae'
                 )
uni_series_mae.head()

In [ ]:
# Train and backtest a model for all items: ForecasterAutoregMultiSeries
items = list(data.columns)

# Define forecaster
window_features = RollingFeatures(stats=['mean', 'min', 'max'], window_sizes=4)
forecaster_ms = ForecasterRecursiveMultiSeries(
                    regressor          = HistGradientBoostingRegressor(random_state=42),
                    lags               = 2,
                    encoding           = 'ordinal',
                    transformer_series = StandardScaler(),
                    window_features    = window_features,
                )
# Backtesting forecaster for all items
cv = TimeSeriesFold(
        steps              = 1,
        initial_train_size = len(data_train) + len(data_val),
        refit              = False,
     )
multi_series_mae, predictions_ms = backtesting_forecaster_multiseries(
                                       forecaster            = forecaster_ms,
                                       series                = data,
                                       levels                = items,
                                       cv                    = cv,
                                       metric                = 'mean_absolute_error',
                                       add_aggregated_metric = False,
                                       verbose               = False,
                                       show_progress         = True
                                   )
# Results
display(multi_series_mae.head(3))
print('')
display(predictions_ms.head(3))

In [ ]:
# Difference of backtesting metric for each item
multi_series_mae = multi_series_mae.set_index('levels')
multi_series_mae.columns = ['multi_series_mae']
results = pd.concat((uni_series_mae, multi_series_mae), axis = 1)
results['improvement'] = results.eval('uni_series_mae - multi_series_mae')
results['improvement_(%)'] = 100 * results.eval('(uni_series_mae - multi_series_mae) / uni_series_mae')
results = results.round(2)

In [ ]:
# Let's plot the gsales_u from the items: ZZUMC, ZYWEM, ZYLQC
items = ['AAXDT', 'ZYWEM', 'ZYLQC']
fig, axs = plt.subplots(3, 1, figsize=(15, 8), sharex=True)
data.loc[:, items].plot(
    legend   = True,
    subplots = True,
    title    = 'Unis sales',
    ax       = axs,
)
for ax in axs:
    ax.axvline(pd.to_datetime(end_train) , color='white', linestyle='--', linewidth=1.5)
    ax.axvline(pd.to_datetime(end_val) , color='white', linestyle='--', linewidth=1.5)
fig.tight_layout()
plt.show()
# Plot predictions from the time series alone and the global model
fig, axs = plt.subplots(3, 1, figsize=(15, 8), sharex=True)

# Plot actual sales
data.loc[:, items].plot(
    legend   = True,
    subplots = True,
    title    = 'Units sales and Predictions',
    ax       = axs,
)

# Plot predictions from the time series alone
for i, item in enumerate(items):
    predictions[item].plot(ax=axs[i], linestyle='--', color='orange', label='Time Series Alone Prediction')

# Plot predictions from the global model
for i, item in enumerate(items):
    predictions_ms[item].plot(ax=axs[i], linestyle='--', color='green', label='Global Model Prediction')

for ax in axs:
    ax.axvline(pd.to_datetime(end_train) , color='white', linestyle='--', linewidth=1.5)
    ax.axvline(pd.to_datetime(end_val) , color='white', linestyle='--', linewidth=1.5)
    ax.legend()

fig.tight_layout()
plt.show()

In [ ]:
# Average improvement for all items
# ======================================================================================
# Let's replace inf in results 'improvement_(%)' with 0 because it means that the model is worse than the time series alone
results = results.replace([np.inf, -np.inf], 0)
# Let's calculate the average mae of both models
average_mae = results.mean()
average_mae = average_mae.round(2)
average_mae

#### By averaging the results of both models, we can see that our performance is better when using a global model. It's also worth mentioning the computational power required to generate this forecast and for the production process.

#### What if we hyper-tune the model? With the help of the library, we can use cv_search to find the best parameters for our model.

In [ ]:
# Hyperparameter search for the multi-series model and backtesting for each item
def search_space(trial):
    search_space  = {
        'lags'          : trial.suggest_categorical('lags', [2, 8]),
        'max_iter'      : trial.suggest_int('max_iter', 100, 500),
        'max_depth'     : trial.suggest_int('max_depth', 2, 7),
        'learning_rate' : trial.suggest_float('learning_rate', 0.01, 0.1)
    }

    return search_space

cv_search = OneStepAheadFold(initial_train_size = len(data_train))

cv_backtesting = TimeSeriesFold(
                        steps              = 2,
                        initial_train_size = len(data_train) + len(data_val),
                        refit              = False,
                        )

window_features = RollingFeatures(stats=['mean', 'min', 'max'], window_sizes=4)
forecaster_ms = ForecasterRecursiveMultiSeries(
                    regressor          = HistGradientBoostingRegressor(random_state=42),
                    lags               = 2,
                    window_features    = window_features,
                    transformer_series = StandardScaler(),
                    encoding           = 'ordinal'
                )

warnings.simplefilter('ignore', category=OneStepAheadValidationWarning)
results_bayesian_ms = bayesian_search_forecaster_multiseries(
                        forecaster    = forecaster_ms,
                        series        = data.loc[:end_val, :],
                        levels        = None, # If is it None select all levels
                        cv            = cv_search,
                        search_space  = search_space,
                        n_trials      = 10,
                        metric        = 'mean_absolute_error',
                        return_best   = True,
                        verbose       = False,
                        show_progress = False
                    )

multi_series_mae, predictions_ms = backtesting_forecaster_multiseries(
                                       forecaster            = forecaster_ms,
                                       series                = data,
                                       levels                = None, # If is it None select all levels
                                       cv                    = cv_backtesting,
                                       metric                = 'mean_absolute_error',
                                       add_aggregated_metric = False,
                                       verbose               = False
                                   )

In [ ]:
# Train and backtest a model for all items: ForecasterAutoregMultiSeries
items = list(data.columns)

# Define forecaster
window_features = RollingFeatures(stats=['mean', 'min', 'max'], window_sizes=4)
forecaster_ms = ForecasterRecursiveMultiSeries(
                    regressor          = HistGradientBoostingRegressor(
                                            max_iter=275,
                                            max_depth=2,
                                            learning_rate=0.04582398297973883,
                                            random_state=42),
                    lags               = [1, 2, 3, 4, 5, 6, 7, 8],
                    encoding           = 'ordinal',
                    transformer_series = StandardScaler(),
                    window_features    = window_features,
                )

# Backtesting forecaster for all items
cv = TimeSeriesFold(
        steps              = 1,
        initial_train_size = len(data_train) + len(data_val),
        refit              = False,
     )
multi_series_mae, predictions_ms = backtesting_forecaster_multiseries(
                                       forecaster            = forecaster_ms,
                                       series                = data,
                                       levels                = items,
                                       cv                    = cv,
                                       metric                = 'mean_absolute_error',
                                       add_aggregated_metric = False,
                                       verbose               = False,
                                       show_progress         = True
                                   )
# Results
display(multi_series_mae.head(3))
print('')
display(predictions_ms.head(3))

In [ ]:
# Difference of backtesting metric for each item
multi_series_mae = multi_series_mae.set_index('levels')
multi_series_mae.columns = ['multi_series_mae']
results_cv = pd.concat((uni_series_mae, multi_series_mae), axis = 1)
results_cv['improvement'] = results.eval('uni_series_mae - multi_series_mae')
results_cv['improvement_(%)'] = 100 * results.eval('(uni_series_mae - multi_series_mae) / uni_series_mae')
results_cv = results_cv.round(2)

In [ ]:
# Average improvement for all items

# Let's replace inf in results 'improvement_(%)' with 0 because it means that the model is worse than the time series alone
results_cv = results_cv.replace([np.inf, -np.inf], 0)
# Let's calculate the average mae of both models
average_mae_cv = results_cv.mean()
average_mae_cv = average_mae_cv.round(2)
average_mae_cv

#### After hyper-tuning the multi-series model, we can see that its performance is better than the previous one. We can now use this model to forecast the sales of the products.

In [ ]:
# Now that we have our best model, let's predict the next 8 weeks

# Define forecaster
window_features = RollingFeatures(stats=['mean', 'min', 'max'], window_sizes=4)

forecaster_ms = ForecasterRecursiveMultiSeries(
                    regressor          = HistGradientBoostingRegressor(
                                            max_iter=275,
                                            max_depth=2,
                                            learning_rate=0.04582398297973883,
                                            random_state=42),
                    lags               = [1, 2, 3, 4, 5, 6, 7, 8],
                    encoding           = 'ordinal',
                    transformer_series = StandardScaler(),
                    window_features    = window_features,
                )

# Fit forecaster
forecaster_ms.fit(data, exog=None)

# Predict next 8 weeks
steps = 8

predictions = forecaster_ms.predict(steps = steps, last_window = data.iloc[-8:, :])

In [ ]:
predictions

In [ ]:
# Let's upload our last 8 week from the csv file

# Load the data
data_8_weeks = pd.read_csv('MasterDataTEST_OP.csv', low_memory=False)

# Convert the date column to datetime
data_8_weeks['cal_date'] = pd.to_datetime(data_8_weeks['cal_date'])

# Filter the data for the last 8 weeks
data_8_weeks = data_8_weeks[data_8_weeks['cal_date'] > '2024-01-01']

# Let's filter the items that we have in our predictions and in the data
items = predictions.columns

data_8_weeks = data_8_weeks[data_8_weeks['sku_id'].isin(items)]

# Pivot the data
data_8_weeks = pd.pivot_table(
                 data    = data_8_weeks,
                 values  = 'gsales_u',
                 index   = 'cal_date',
                 columns = 'sku_id'
             )

In [ ]:
# let's calculate the MAE with the actual sales from data_8_weeks and the predictions
mae = np.abs(data_8_weeks - predictions).mean().mean()

print(f"Mean Absolute Error for the next 8 weeks: {mae:.2f}")


#### Great our model performed better than our benchmark and without including the exogenous variables.


#### Let's plot some of our items to compare the sales

In [ ]:
# Let's plot some of the predictions
items = np.random.choice(data_8_weeks.columns, 5)

fig, axs = plt.subplots(5, 1, figsize=(15, 8), sharex=True)

for i, item in enumerate(items):
    data_8_weeks[item].plot(ax=axs[i], color='green', label='Actual')
    predictions[item].plot(ax=axs[i], linestyle='--', color='orange', label='Predicted')
    axs[i].set_title(item)
    axs[i].legend()

plt.show()

#### As we can observe in the chart, our model is generally able to capture the sales trend. There is significant room for improvement in the forecast, and more feature engineering can be tested, but the results from this multi-series model provide a good starting point for our client.

#### What I always like to tell them is: they can focus on analyzing the products and the forecast output to identify those that weren't captured well by the model and make adjustments, as they have more knowledge of the operations and suppliers.